In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
print(tf.__version__)

2.1.0


In [3]:
if tf.test.gpu_device_name():
   print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [205]:
path_to_file = 'C:/Users/vaibh/Documents/shikata/Untitled Folder/wayne.txt'

In [206]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 117290 characters


In [207]:
# Take a look at the first 250 characters in text
print(text[:250])


﻿They call me Mr Carter I kissed the daughter
Of the deads forehead I killed the father
Spilled the heart of a mildew hater
I will put them body on chill like glaciers
Gracias Im crazy yes its obvious
Going against me is atheist
I got my angels


In [208]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))


57 unique characters


In [209]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])


In [210]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')


{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  "'" :   3,
  'A' :   4,
  'B' :   5,
  'C' :   6,
  'D' :   7,
  'E' :   8,
  'F' :   9,
  'G' :  10,
  'H' :  11,
  'I' :  12,
  'J' :  13,
  'K' :  14,
  'L' :  15,
  'M' :  16,
  'N' :  17,
  'O' :  18,
  'P' :  19,
  ...
}


In [211]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:17]), text_as_int[:17]))


'\ufeffThey call me Mr ' ---- characters mapped to int ---- > [56 23 37 34 54  2 32 30 41 41  2 42 34  2 16 47  2]


In [212]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])


﻿
T
h
e
y


In [213]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))


'\ufeffThey call me Mr Carter I kissed the daughter\r\nOf the deads forehead I killed the father\r\nSpilled the'
' heart of a mildew hater\r\nI will put them body on chill like glaciers\r\nGracias Im crazy yes its obvio'
'us\r\nGoing against me is atheist\r\nI got my angels on my shoulders and a quarter of that angel dust\r\nI '
'aint sniffin Im just pitchin ya honor I aint snitchin ya honor\r\nHate bitch niXgas bitches with power\r'
'\nVacate when the kitchen get hotter\r\nI just sit on the counter open the cabinet close the cupboard\r\nP'


In [214]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text


dataset = sequences.map(split_input_target)


In [215]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))


Input data:  '\ufeffThey call me Mr Carter I kissed the daughter\r\nOf the deads forehead I killed the father\r\nSpilled th'
Target data: 'They call me Mr Carter I kissed the daughter\r\nOf the deads forehead I killed the father\r\nSpilled the'


In [216]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))


Step    0
  input: 56 ('\ufeff')
  expected output: 23 ('T')
Step    1
  input: 23 ('T')
  expected output: 37 ('h')
Step    2
  input: 37 ('h')
  expected output: 34 ('e')
Step    3
  input: 34 ('e')
  expected output: 54 ('y')
Step    4
  input: 54 ('y')
  expected output: 2 (' ')


In [217]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [218]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024


In [255]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [220]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [256]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 57) # (batch_size, sequence_length, vocab_size)


In [257]:
model.summary()


Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (64, None, 256)           14592     
_________________________________________________________________
gru_28 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_28 (Dense)             (64, None, 57)            58425     
Total params: 4,011,321
Trainable params: 4,011,321
Non-trainable params: 0
_________________________________________________________________


In [258]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([25, 21, 25, 29, 16, 45, 19, 42, 10, 37, 10,  9, 28, 53, 31, 55, 21,
       36, 27, 54, 43, 27, 40,  6, 46,  7, 27, 29, 28, 48, 15, 15,  7, 37,
       54,  7, 30, 36, 40, 12, 53, 37, 40, 44, 45, 29, 33, 17, 23, 10, 40,
        5,  9, 16, 10, 14, 39, 10, 26, 38, 25, 46, 41, 28, 26, 26,  2, 32,
       18, 30, 28, 14,  4, 48, 48, 24, 35, 47,  3, 27, 48, 27, 49,  4,  2,
       42, 10, 40, 34, 37,  9, 22, 37,  6, 26, 17, 47, 14, 51,  6],
      dtype=int64)

In [259]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'stroke my shit break brooms\r\nI been high sinceniXga\r\nThatsdegrees Nick Lachey niXga\r\nHope you know t'

Next Char Predictions: 
 "VRVZMpPmGhGFYxbzRgXynXkCqDXZYsLLDhyDagkIxhkopZdNTGkBFMGKjGWiVqlYWW cOaYKAssUfr'XsXtA mGkehFShCWNrKvC"


In [260]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())


Prediction shape:  (64, 100, 57)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.0423827


In [262]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss=loss)

In [263]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [228]:
EPOCHS=25

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Train for 18 steps
Epoch 1/25
18/18 [==============================] - 6s 318ms/step - loss: 3.9740
Epoch 2/25
18/18 [==============================] - 2s 129ms/step - loss: 3.0616
Epoch 3/25
18/18 [==============================] - 2s 130ms/step - loss: 2.6946
Epoch 4/25
18/18 [==============================] - 2s 134ms/step - loss: 2.4301
Epoch 5/25
18/18 [==============================] - 2s 127ms/step - loss: 2.3019 2s 
Epoch 6/25
18/18 [==============================] - 2s 127ms/step - loss: 2.2347
Epoch 7/25
18/18 [==============================] - 2s 128ms/step - loss: 2.1832 1s - l
Epoch 8/25
18/18 [==============================] - 2s 126ms/step - loss: 2.1348 0s - loss:
Epoch 9/25
18/18 [==============================] - 2s 125ms/step - loss: 2.0851 0s - loss: 2.088
Epoch 10/25
18/18 [==============================] - 2s 129ms/step - loss: 2.0366
Epoch 11/25
18/18 [==============================] - 2s 130ms/step - loss: 1.9896
Epoch 12/25
18/18 [==============================

In [266]:
tf.train.latest_checkpoint(checkpoint_dir)


'./training_checkpoints\\ckpt_39'

In [267]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))


In [268]:
model.summary()



Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (1, None, 256)            14592     
_________________________________________________________________
gru_29 (GRU)                 (1, None, 1024)           3938304   
_________________________________________________________________
dense_29 (Dense)             (1, None, 57)             58425     
Total params: 4,011,321
Trainable params: 4,011,321
Non-trainable params: 0
_________________________________________________________________


In [269]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))


In [ ]:
print(generate_text(model, start_string=u"Start "))


In [253]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)


In [254]:
optimizer = tf.keras.optimizers.Adam()


In [264]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [265]:
# Training step
EPOCHS = 40

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))


Epoch 1 Batch 0 Loss 4.042327404022217
Epoch 1 Loss 3.6895
Time taken for 1 epoch 5.071139574050903 sec

Epoch 2 Batch 0 Loss 3.636118173599243
Epoch 2 Loss 2.8645
Time taken for 1 epoch 1.2915172576904297 sec

Epoch 3 Batch 0 Loss 2.903195381164551
Epoch 3 Loss 2.5823
Time taken for 1 epoch 1.2292766571044922 sec

Epoch 4 Batch 0 Loss 2.511789560317993
Epoch 4 Loss 2.3388
Time taken for 1 epoch 1.2732868194580078 sec

Epoch 5 Batch 0 Loss 2.395855665206909
Epoch 5 Loss 2.2612
Time taken for 1 epoch 1.7180371284484863 sec

Epoch 6 Batch 0 Loss 2.2354798316955566
Epoch 6 Loss 2.2155
Time taken for 1 epoch 1.2641708850860596 sec

Epoch 7 Batch 0 Loss 2.1939737796783447
Epoch 7 Loss 2.1894
Time taken for 1 epoch 1.2661690711975098 sec

Epoch 8 Batch 0 Loss 2.189877510070801
Epoch 8 Loss 2.1072
Time taken for 1 epoch 1.2853343486785889 sec

Epoch 9 Batch 0 Loss 2.081739664077759
Epoch 9 Loss 2.0920
Time taken for 1 epoch 1.276651382446289 sec

Epoch 10 Batch 0 Loss 2.038172960281372
Epoch 

Epoch 33 Batch 0 Loss 1.0050543546676636
Epoch 33 Loss 1.0387
Time taken for 1 epoch 1.2392961978912354 sec

Epoch 34 Batch 0 Loss 0.9360613822937012
Epoch 34 Loss 0.9753
Time taken for 1 epoch 1.2507188320159912 sec

Epoch 35 Batch 0 Loss 0.8599122762680054
Epoch 35 Loss 0.9590
Time taken for 1 epoch 1.7102892398834229 sec

Epoch 36 Batch 0 Loss 0.8637584447860718
Epoch 36 Loss 0.8483
Time taken for 1 epoch 1.3002939224243164 sec

Epoch 37 Batch 0 Loss 0.7560678124427795
Epoch 37 Loss 0.7920
Time taken for 1 epoch 1.2832906246185303 sec

Epoch 38 Batch 0 Loss 0.679833173751831
Epoch 38 Loss 0.7737
Time taken for 1 epoch 1.271287441253662 sec

Epoch 39 Batch 0 Loss 0.6289311647415161
Epoch 39 Loss 0.7120
Time taken for 1 epoch 1.2332775592803955 sec

Epoch 40 Batch 0 Loss 0.5969004034996033
Epoch 40 Loss 0.6278
Time taken for 1 epoch 1.7095251083374023 sec

